In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
%cd /path_to_code/SHAnTIE/
import astropy.units as u

In [ ]:
from astropy.io import fits
import numpy as np
import healpy as hp
import h5py
import copy

In [ ]:
from intensity_mapping import rebin_galaxies as rg

In [ ]:
#Select real galaxies to create the data file, or random galaxies to create a selection function.
path = 'path_to_BOSS_galaxy_catalog_files'
real_not_sims = True
if real_not_sims:
    fname = path + 'galaxy_DR12v5_CMASS_North.fits.gz'
else:
    fname = path + 'random1_DR12v5_CMASS_North.fits.gz'
CMASS_North_real_fits = fits.open(fname)

#Select edges of z-bins to use here. This example uses bins of size 0.2.
z_left = np.array([0,0.2,0.4,0.6,0.8])
z_right = np.array([0.2,0.4,0.6,0.8,1])
#Set Nside for output binned catalog.
Nside = 512

In [ ]:
print(CMASS_North_real_fits.info())
print(list(CMASS_North_real_fits[0].header.keys()))
print(vars(CMASS_North_real_fits[1]))

In [ ]:
print(CMASS_North_real_fits[1].data['RA'])
print(CMASS_North_real_fits[1].data['dec'])
print(CMASS_North_real_fits[1].data['z'])

print(CMASS_North_real_fits[1].data['WEIGHT_SYSTOT'])

In [ ]:
#Define function to make hitmap.

def make_hitmap_from_catalog_file(fname, nside, z_left, z_right, output_coord = 'G', apply_wtot = False):
    with fits.open(fname) as f:
        ra_deg =  f[1].data['RA']
        dec_deg = 90 - f[1].data['DEC']
        zs = f[1].data['z']
    if not apply_wtot:
        return rg.bin_catalog2d(ra_deg*u.deg, dec_deg*u.deg, zs, nside, z_left, z_right, coord = output_coord)
    else:
        wsystot = np.array(f[1].data['WEIGHT_SYSTOT'])
        wcp = np.array(f[1].data['WEIGHT_CP'])
        wnoz = np.array(f[1].data['WEIGHT_NOZ'])
        wtot = wsystot*(wcp + wnoz - 1)
        return rg.bin_catalog2d(ra_deg*u.deg, dec_deg*u.deg, zs, nside, z_left, z_right, weights = wtot,
                                coord = output_coord)

def make_hitmap_N_and_S(path, file_beginning, nside, z_left, z_right, output_coord = 'G', file_ending = '.fits.gz',
                       apply_wtot = False):
    full_path = path + file_beginning + 'North' + file_ending
    north_hits = make_hitmap_from_catalog_file(full_path, nside, z_left, z_right, output_coord = output_coord,
                                              apply_wtot = apply_wtot)
    full_path = path + file_beginning + 'South' + file_ending
    south_hits = make_hitmap_from_catalog_file(full_path, nside, z_left, z_right, output_coord = output_coord,
                                              apply_wtot = apply_wtot)
    ans = [north_hits[0] + south_hits[0], north_hits[1] + south_hits[1]]
    return ans

def save_hitmap(path, file_beginning, nside, z_left, z_right, output_fname, apply_wtot = False,
                output_coord = 'G', file_ending = '.fits.gz'):
    hitmaps = make_hitmap_N_and_S(path, file_beginning, nside, 
                                  z_left, z_right, output_coord = output_coord, file_ending = file_ending)
    z_edges = np.zeros((len(z_left) + 1))
    z_edges[:-1] = z_left
    z_edges[-1] = z_right[-1]
    if apply_wtot:
        wtot_weighted_hitmaps = make_hitmap_N_and_S(path, file_beginning, nside, 
                                  z_left, z_right, apply_wtot = True, 
                                                    output_coord = output_coord, file_ending = file_ending)
    with h5py.File(output_fname, 'a') as f:
        f['z_edges'] = z_edges
        f['num_gals_per_z'] = hitmaps[0]
        f['hitmap'] = hitmaps[1].T
        if apply_wtot:
            f['wtot_weighted_hitmap'] = wtot_weighted_hitmaps[1].T

In [ ]:
#Save binned galaxy counts.
if real_not_sims:
    save_hitmap(path, 'galaxy_DR12v5_CMASS_', Nside, z_left, z_right, 
            path + 'number_counts_3d/Nside_512/CMASS_real_galaxies_with_wtot_weighting.h5', apply_wtot=True)
else:
    save_hitmap(path, 'random1_DR12v5_CMASS_', Nside, z_left, z_right, 
            path + 'number_counts_3d/Nside_512/CMASS_random1_galaxies_dzp1.h5', apply_wtot=False)
